## Correlation between news and stock movement

Load sentiment.csv and stock_data.csv datasets
- merge together with the same date

In [1]:
import pandas as pd

# read the datasets
sentiment_df = pd.read_csv('../Data/sentiment.csv')
stock_df = pd.read_csv('../Data/stock_data.csv')